# Deploying and triggering Flex template runs

In [1]:
import json
from datetime import datetime
import time

## Build the Flex template docker image

In [6]:
template_name = 'drift-detector'

project_id = !(gcloud config get-value core/project)
project_id = project_id[0]
template_image = 'gcr.io/{}/{}:latest'.format(project_id, template_name)

!gcloud builds submit --tag {template_image} .

3,>=2.17->tensorflow_data_validation==0.21.5) (2019.3)
  Created wheel for absl-py: filename=absl_py-0.8.1-py3-none-any.whl size=121165 sha256=e90b0f411ff8fb865559651911619aab37a65091cd1a62aefe34a03bad7dd040
  Stored in directory: /root/.cache/pip/wheels/0c/fa/af/c3217c654740d2cffc40ab4eca9da558be4bb94dd593ef0d74
  Created wheel for tensorflow-transform: filename=tensorflow_transform-0.21.2-py3-none-any.whl size=301093 sha256=4619e414b7f8908b043972aa01f091b07e223c0de353b0b11df2be1754eda08c
  Stored in directory: /root/.cache/pip/wheels/a6/9c/9a/a0dbfc0948873609990f4360abb0d1a4fb0960ab5cde371a6c
  Created wheel for backcall: filename=backcall-0.1.0-py3-none-any.whl size=10413 sha256=c2829adf38f079d2dbee62181eb0706332f845289e648d6636c68a7fe7a3c240
  Stored in directory: /root/.cache/pip/wheels/cb/6f/5c/9b66a87adeb46fed2babb14407bae1271f5e41b6b34b642411
Successfully built absl-py tensorflow-transform backcall
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.9.0
 

## Deploy the Flex template using `gcloud`

In [3]:
flex_templates_location = 'gs://mlops-dev-workspace/flex-templates'
template_path = '{}/{}.json'.format(flex_templates_location, template_name)
metadata_file = 'metadata.json'

!gcloud beta dataflow flex-template build {template_path} \
  --image {template_image} \
  --sdk-language "PYTHON" \
  --metadata-file {metadata_file}

Successfully saved container spec in flex template file.
Template File GCS Location: gs://mlops-dev-workspace/flex-templates/drift-detector.json
Container Spec:

{
    "image": "gcr.io/mlops-dev-env/drift-detector:latest",
    "metadata": {
        "description": "Data drift detector Python flex template.",
        "name": "Data drift detector Python flex template",
        "parameters": [
            {
                "helpText": "A full name of the BQ request-response log table",
                "label": "Request response log table.",
                "name": "request_response_log_table",
                "regexes": [
                    "[-_.a-zA-Z0-9]+"
                ]
            },
            {
                "helpText": "A type of instances in request_response log_table",
                "label": "Instance type.",
                "name": "instance_type",
                "regexes": [
                    "LIST|OBJECT"
                ]
            },
            {
              

## Trigger a run of the template using REST API

In [4]:
service_url = 'https://dataflow.googleapis.com/v1b3/projects/{}/locations/us-central1/flexTemplates:launch'.format(project_id)
headers_content = 'Content-Type: application/json'
access_token = !(gcloud auth print-access-token) 
headers_auth = 'Authorization: Bearer {}'.format(access_token[0])
parameters = {
    'request_response_log_table': 'mlops-dev-env.data_validation.covertype_classifier_logs_tf',
    'instance_type': 'OBJECT',
    'start_time': '2020-05-15T00:15:00',
    'end_time': '2020-05-15T05:51:00',
    'output_path': 'gs://mlops-dev-workspace/drift_monitor/output/tf',
    'schema_file': 'gs://mlops-dev-workspace/drift_monitor/schema/schema.pbtxt',
    'setup_file': './setup.py',

}
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
body = {
    'launch_parameter': 
         {
             'jobName': job_name,
             'parameters' : parameters,
             'containerSpecGcsPath': template_path
         }}

json_body = json.dumps(body)

In [5]:
!curl -X POST \
  "{service_url}" \
  -H "{headers_content}" \
  -H "{headers_auth}" \
  -d '{json_body}'

{
  "job": {
    "id": "2020-05-17_09_33_20-13568616329705819873",
    "projectId": "mlops-dev-env",
    "name": "data-drift-20200517-163317",
    "currentStateTime": "1970-01-01T00:00:00Z",
    "createTime": "2020-05-17T16:33:21.172970Z",
    "location": "us-central1",
    "startTime": "2020-05-17T16:33:21.172970Z"
  }
}


## Trigger a run of the template using `gcloud`

In [ ]:
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
g_parameters = ','.join(['{}={}'.format(key,value) for key, value in parameters.items()])

!gcloud beta dataflow flex-template run {job_name} \
--template-file-gcs-location {template_path} \
--parameters {g_parameters}